In [ ]:
pip install tensorflow keras matplotlib numpy opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 357.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 MB 3.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 2.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 5.8 MB/s eta 0

In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

2024-07-23 14:48:56.988558: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
import os
import csv

def create_labels(image_folder, label):
    # Check if the file is empty to write header
    file_exists = os.path.isfile('labels.csv')
    
    with open('labels.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header if the file is empty
        if not file_exists or os.path.getsize('labels.csv') == 0:
            writer.writerow(['image', 'label'])
        
        for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                writer.writerow([filename, label])

# Paths to your image folders
solar_farm_folder = '/Users/macbookpro/Desktop/SolarProject/SolarFarm_Images'
non_solar_farm_folder = '/Users/macbookpro/Desktop/SolarProject/Non_SolarFarms'

# Create labels
create_labels(solar_farm_folder, 1)  # Label 1 for solar farms
create_labels(non_solar_farm_folder, 0)  # Label 0 for non-solar farms


In [4]:
base_dir = '/Users/macbookpro/Desktop/SolarProject/dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Create directories if they don't exist
os.makedirs(os.path.join(train_dir, 'solar_farm'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'non_solar_farm'), exist_ok=True)
os.makedirs(os.path.join(validation_dir, 'solar_farm'), exist_ok=True)
os.makedirs(os.path.join(validation_dir, 'non_solar_farm'), exist_ok=True)


In [20]:
df = pd.read_csv('labels.csv')
# Define function to split images into train and validation
# Paths to your image folders
solar_farm_folder = '/Users/macbookpro/Desktop/SolarProject/SolarFarm_Images'
non_solar_farm_folder = '/Users/macbookpro/Desktop/SolarProject/Non_SolarFarms'
def split_and_copy_images(df, img_folder, output_train_folder, output_validation_folder, test_size=0.2):
    # Split dataframe
    train_df, val_df = train_test_split(df, test_size=test_size, stratify=df['label'])
    
    # Copy images to respective directories
    for index, row in train_df.iterrows():
        src_path = os.path.join(img_folder, row['image'])
        # dest_path = os.path.join(output_train_folder, 'solar_farm' if row['label'] == 1 else 'non_solar_farm', row['image'])
        dest_path = output_train_folder
        shutil.copy(src_path, dest_path)
    
    for index, row in val_df.iterrows():
        src_path = os.path.join(img_folder, row['image'])
        #dest_path = os.path.join(output_validation_folder, 'solar_farm' if row['label'] == 1 else 'non_solar_farm', row['image'])
        dest_path = output_validation_folder
        shutil.copy(src_path, dest_path)

# Split and copy images
split_and_copy_images(df[df['label'] == 1], solar_farm_folder, os.path.join(train_dir, 'solar_farm'), os.path.join(validation_dir, 'solar_farm'))
split_and_copy_images(df[df['label'] == 0], non_solar_farm_folder, os.path.join(train_dir, 'non_solar_farm'), os.path.join(validation_dir, 'non_solar_farm'))

In [21]:
image_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 83 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [22]:
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = create_cnn_model()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.6424 - loss: 0.6419 - val_accuracy: 0.7727 - val_loss: 0.5593
Epoch 2/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.7188 - loss: 0.6330

2024-07-23 15:15:31.916094: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 988ms/step - accuracy: 0.7188 - loss: 0.6330 - val_accuracy: 0.5909 - val_loss: 0.6497
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.4671 - loss: 0.7586 - val_accuracy: 0.7727 - val_loss: 0.5212
Epoch 4/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step - accuracy: 0.7188 - loss: 0.5768

2024-07-23 15:15:35.896064: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.7188 - loss: 0.5768 - val_accuracy: 0.7727 - val_loss: 0.5178
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7468 - loss: 0.5744 - val_accuracy: 0.5000 - val_loss: 0.6707
Epoch 6/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 509ms/step - accuracy: 0.4062 - loss: 0.8108

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 996ms/step - accuracy: 0.6875 - loss: 0.5602 - val_accuracy: 0.6818 - val_loss: 0.5704
Epoch 17/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5982 - loss: 0.6330 - val_accuracy: 0.7273 - val_loss: 0.4296
Epoch 18/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - accuracy: 0.8125 - loss: 0.4413

2024-07-23 15:16:04.539584: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step - accuracy: 0.8125 - loss: 0.4413 - val_accuracy: 0.8182 - val_loss: 0.3775
Epoch 19/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7207 - loss: 0.5623 - val_accuracy: 0.8182 - val_loss: 0.3794
Epoch 20/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.7188 - loss: 0.5633

2024-07-23 15:16:08.696815: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 994ms/step - accuracy: 0.7188 - loss: 0.5633 - val_accuracy: 0.7273 - val_loss: 0.5083


In [25]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 994ms/step - accuracy: 0.7273 - loss: 0.5083
Test accuracy: 0.7272727489471436
